# Parameter sweep for the puncta analysis

## Requirements
- A folder with images that should be analyzed.  All z-layers for a specific sample must be combined into a single file. To combine z-layers and channels, run [run_images_to_stack.ipynb](run_images_to_stack.ipynb). 

- To compute puncta statistics per cell, cell segmentation should be provided as an additional channel. To segment cells/nuclei, run [run_cell_segmentation.ipynb](run_cell_segmentation.ipynb).

<hr style="height:2px;">

## Config

<hr style="height:2px;">

### The following code imports and declares functions used for the processing:

In [ ]:
#################################
#  Don't modify the code below  #
#################################

import json
import os
import itertools
import pandas as pd 
import copy
from am_utils.utils import combine_statistics
from punctatools.lib.segment import segment_puncta_batch
from punctatools.lib.quantify import quantify_batch
from punctatools.lib.utils import load_parameters

### Specify the parameter file with the default parameter values


In [ ]:
parameter_file = 'parameters.json'

### Specidy the output directory for the parameter sweep

In [ ]:
output_dir = '../parameter_sweep'


### The following code loads the parameters 

In [ ]:
#################################
#  Don't modify the code below  #
#################################

with open(parameter_file) as f:
    kwargs = json.load(f)

kwargs['puncta_analysis_dir'] = os.path.realpath(output_dir)
kwargs

### Please specify the parameter values you would like to sweep over

In [ ]:
threshold_detection = [0.003, 0.002]
threshold_segmentation = [0.001]
threshold_background = [3]

<hr style="height:2px;">

## Processing

<hr style="height:2px;">

### The following code prints all parameter combinations

In [ ]:
#################################
#  Don't modify the code below  #
#################################


sweep_vars = []
sweep_values = []

cur_vars = vars().copy()
for var in cur_vars.keys():
    if var in kwargs and var != 'output_dir':
        sweep_vars.append(var)
        sweep_values.append(cur_vars[var])
        
print('Parameters to sweep over')
for k, v in zip(sweep_vars, sweep_values):
    print(rf"{k}: {v}")

combinations = list(itertools.product(*sweep_values))

print(f'\nNumber of combinations: {len(combinations)}:')
for c in combinations:
    print(c)
    
print('\nDoes this look correct? If not, go back and specify the parameter values to sweep over')

### The following code runs the batch analysis for the given parameter combinations

In [ ]:
#################################
#  Don't modify the code below  #
#################################

outputs = []

for vals in combinations:
    print('Parameter values:', vals)
    kwargs_new = kwargs.copy()
    fn_out = ''
    for k, v in zip(sweep_vars, vals):
        kwargs_new[k] = v
        fn_out += rf"{k}={v}_"
        
    fn_out = fn_out.rstrip('_')
    kwargs_new['puncta_analysis_dir'] = os.path.join(kwargs['puncta_analysis_dir'], fn_out)
    outputs.append(kwargs_new['puncta_analysis_dir'])
    p_file = os.path.realpath(os.path.join(kwargs_new['puncta_analysis_dir'], 'parameters.json'))
    print('Output_dir:', fn_out)
    
    os.makedirs(kwargs_new['puncta_analysis_dir'], exist_ok=True)
    
    with open(p_file, 'w') as f:
        json.dump(kwargs_new, f, indent=4)
        
    print('Run the analysis...')
        
    command = rf'python ../scripts/run_puncta_analysis.py -p {p_file}'
    os.system(command)
    
    print('\n')
        

### The following code combines statistics

In [ ]:
#################################
#  Don't modify the code below  #
#################################

cell_stat_dr = kwargs['cell_stat_dir']
puncta_stat_dr = kwargs['puncta_stat_dir']

stats = [pd.DataFrame()]*2

for vals, op in zip(combinations, outputs):
    for i, stat_dir in enumerate([cell_stat_dr, puncta_stat_dr]):
        fn = os.path.join(op, stat_dir + '.csv')
        df = pd.read_csv(fn)
        
        for k, v in zip(sweep_vars, vals):
            df[k] = v
        stats[i] = pd.concat([stats[i], df], ignore_index=True)
        
for i, stat_dir in enumerate([cell_stat_dr, puncta_stat_dr]):
    stats[i].to_csv(os.path.join(kwargs['puncta_analysis_dir'], stat_dir + '.csv'))